In [1]:
import os
import sys
print(os.path.abspath(''))

pkg_path = os.path.abspath('..')
print(pkg_path)
sys.path.insert(0, pkg_path)

import pandas
corrected_data = pandas.read_csv('../data/log_normal_fitted.csv')

/home/campus.ncl.ac.uk/b8051106/my/src/python/pyolin/notebooks
/home/campus.ncl.ac.uk/b8051106/my/src/python/pyolin


In [2]:
corrected_data

,filename,date,strain,plasmid,backbone,iptg_concentration,lowfsc,lowgfp,real_time,log_mean_gfp,log_mean_v,log_std_gfp,log_std_v,log_rho,fit_goodness,std_gfp_correct
0,Huseyin2019-09-24.0001.fcs,2019-09-24,KT 2440,1201,pSeva221,0,6.425437,0.608937,2019-09-24 15:55:05,0.250036,2.314908,0.474749,0.963332,0.299708,0.975201,0.452925
1,Huseyin2019-09-24.0002.fcs,2019-09-24,KT 2440,1201,pSeva221,5,6.015331,0.539061,2019-09-24 15:56:06,0.037474,1.950081,0.456776,0.945077,0.328764,0.989436,0.431385
2,Huseyin2019-09-24.0003.fcs,2019-09-24,KT 2440,1201,pSeva221,10,6.055118,0.552027,2019-09-24 15:57:07,0.093799,2.196333,0.463687,0.949740,0.289366,0.982129,0.443850
3,Huseyin2019-09-24.0004.fcs,2019-09-24,KT 2440,1201,pSeva221,20,7.181627,0.531118,2019-09-24 15:58:08,0.173098,2.280760,0.494051,0.984647,0.353470,0.981735,0.462158
4,Huseyin2019-09-24.0005.fcs,2019-09-24,KT 2440,1201,pSeva221,30,6.007504,0.523445,2019-09-24 15:59:09,0.044898,2.189524,0.461442,0.961913,0.282821,0.979128,0.442603
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
848,Huseyin2019-09-26.0281.fcs,2019-09-26,DH5α,SrpR-S4,pSeva221,200,4.757235,0.414770,2019-09-26 15:48:57,-0.194275,2.421361,0.465537,1.088471,0.171667,0.983499,0.458626
849,Huseyin2019-09-26.0282.fcs,2019-09-26,DH5α,SrpR-S4,pSeva221,500,5.140736,0.430098,2019-09-26 15:49:59,-0.123461,2.671281,0.490201,1.087739,0.196164,0.978036,0.480677
850,Huseyin2019-09-26.0283.fcs,2019-09-26,DH5α,SrpR-S4,pSeva221,1000,5.105510,0.435157,2019-09-26 15:51:01,-0.105915,2.627236,0.493216,1.106167,0.191733,0.979958,0.484066
851,Huseyin2019-09-26.0284.fcs,2019-09-26,DH5α,SrpR-S4,pSeva221,2000,4.655399,0.431738,2019-09-26 15:52:03,-0.109492,2.818970,0.488869,1.133579,0.219406,0.973108,0.476957


In [3]:
def get_all_plasmid(plasmid_id, df):
    return df[(df.plasmid == plasmid_id)]
    
def get_mean_gfps(plasmid_id, strain_id, backbone_id, df):
    data = df[(df.plasmid == plasmid_id) & (df.backbone == backbone_id) & (df.strain == strain_id)]
    return [point for point in zip(data.iptg_concentration, data.log_mean_gfp)]

def get_plasmids(df):
    return df.plasmid.drop_duplicates()[df.plasmid != 'Empty']

def get_strains(df):
    return df.strain.drop_duplicates()[df.strain != 'Empty']

def get_backbones(df):
    return df.backbone.drop_duplicates()[df.backbone != 'Empty']

get_plasmids(corrected_data)
get_strains(corrected_data)
get_backbones(corrected_data)

0      pSeva221
288         pAN
Name: backbone, dtype: object

In [4]:
from pyolin.gate import Gate
from pyolin.utils import shape_agreement

parts = {}

for plasmid in get_plasmids(corrected_data):
    gates = []
    for strain in get_strains(corrected_data):
        for backbone in get_backbones(corrected_data):
            characterisation = get_mean_gfps(plasmid, strain, backbone, corrected_data)
            if characterisation:
                xs = [x[0] for x in characterisation]
                ys = [y[1] for y in characterisation]
                gates.append(Gate(f"{strain}_{backbone}_{plasmid}", xs, ys))
            
    parts[plasmid] = gates

TypeError: __init__() missing 1 required positional argument: 'rpu_out'

In [ ]:
from pyolin.utils import similarity

import similaritymeasures as sm

for part in parts:
    gates = parts[part]
    for a in gates:
        for b in gates:
            score = similarity(a, b)
            print(f"{a.name} vs. {b.name} = {score}")

In [ ]:
for part in parts:
    gates = parts[part]
    for a in gates:
        for b in gates:
            shape_score = shape_agreement(a, b)
            translation_score = abs(a.params["ymin"] - b.params["ymin"]) + abs(a.params["ymax"] - b.params["ymax"]) 
            print(f"{a.name} vs. {b.name} = {shape_score + translation_score}")

In [5]:
from FlowCytometryTools import FCMeasurement
from FlowCytometryTools import FCPlate
from FlowCytometryTools import PolyGate

sample = FCMeasurement(ID='Test', datafile='../data/FCS/Huseyin2019-10-04.0114.fcs')

/home/campus.ncl.ac.uk/b8051106/src/anaconda3/lib/python3.7/site-packages/FlowCytometryTools/core/docstring.py:56: MatplotlibDeprecationWarning: 
The dedent function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use inspect.cleandoc instead.
  doc = dedent(doc)


In [6]:
sample.get_meta()

{'__header__': {'FCS format': b'FCS3.1',
  'text start': 58,
  'text end': 2498,
  'data start': 2499,
  'data end': 7451370,
  'analysis start': 0,
  'analysis end': 0},
 '$FIL': 'export_Huseyin2019-10-04.0114.mqd.fcs',
 '$TOT': 80966,
 '$PAR': 23,
 '$BYTEORD': '1,2,3,4',
 '$DATATYPE': 'F',
 'FJ_FCS_VERSION': '3.1',
 '$EXP': 'admin',
 '$SYS': 'MACSQuantify,2.8.1618.16380',
 '$CYT': 'MACSQuant VYB,2.8.1618.16380',
 '$CYTSN': '3090',
 '$SRC': '239-247',
 '$CELLS': 'KT2440 pSEVA251',
 '$DATE': '2019-Oct-04',
 '$BTIM': '17:05:31',
 '$ETIM': '17:05:58',
 '$PROJ': 'Huseyin',
 '$TIMESTEP': '1',
 '$LAST_MODIFIED': '2019-Oct-04 17:05:58',
 '$LAST_MODIFIER': 'admin',
 '$ORIGINALITY': 'Original',
 '$WELLID': 'B2',
 '$TR': 'FSC,0',
 '$ABRT': '0',
 '$LOST': '0',
 '$VOL': '2000',
 '$MODE': 'L',
 '$NEXTDATA': 0,
 '$BEGINANALYSIS': '0',
 '$ENDANALYSIS': '0',
 '$BEGINSTEXT': '8033244',
 '$ENDSTEXT': '8628092',
 '$BEGINDATA': '2499',
 '$ENDDATA': '7451370',
 '_channels_':                        $PnS   

In [ ]:
sample.transform('tlog', channels=['GFP-A', 'GFP-H']).plot(['GFP-A', 'GFP-H']);